## Install dlib

In [0]:
! apt install python python-pip build-essential cmake pkg-config libx11-dev libatlas-base-dev libgtk-3-dev libboost-python-dev -y
!pip install dlib

## install face_recognition 

In [0]:
! pip3 install face_recognition

In [1]:
# check for dlib on gpu
import dlib.cuda as cuda
print(cuda.get_num_devices())

1


In [0]:
import face_recognition
import cv2
import glob
import numpy as np
import os
import joblib

In [0]:
files_path = "./face_recognition/"

## Train Model

In [0]:
from sklearn import svm

In [0]:
# The training data would be all the face encodings from all the known images and the labels are their folder names
encodings = []
names = []

In [0]:
# Training directory
train_dir = os.listdir(files_path+'train_dir/')

In [8]:
train_dir

['Ben-Affleck',
 'Henry-Cavill',
 'Json-Mamoa',
 'Ezra-Miller',
 'Ray-Fisher',
 'Gal-Gadot']

In [0]:
# Loop through each person in the training directory
for person in train_dir:
    pix = os.listdir(files_path+'train_dir/' + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file(files_path+'train_dir/' + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face,model="cnn") # model ='cnn' uses gpu model = 'HOG' on cpu

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

In [10]:
# Training the SVC classifier
clf = svm.SVC(gamma='scale',probability=True)
clf.fit(encodings,names)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
# Load the test image with faces into a numpy array
test_image = face_recognition.load_image_file(files_path+'train_dir/Gal-Gadot/g1.jpg')

In [22]:
# Find all the faces in the test image using the default HOG-based model
face_locations = face_recognition.face_locations(test_image)
no = len(face_locations)
print("Number of faces detected: ", no)

Number of faces detected:  1


In [23]:
# Predict all the faces in the test image using the trained classifier
print("Found:")
for i in range(no):
    test_image_enc = face_recognition.face_encodings(test_image)[i]
    name = clf.predict([test_image_enc])
    prob = clf.predict_proba([test_image_enc])
    print(*name)
    print(prob)

Found:
Gal-Gadot
[[0.08361542 0.08434837 0.66973355 0.05800135 0.05216002 0.0521413 ]]


In [0]:
filename = files_path+'face_recognition_svm.pkl'

In [15]:
# save trained model
joblib.dump(clf, filename)

['/content/drive/My Drive/DLCP/openwork/face_recognition/justice_league/face_recognition_svm.pkl']

In [0]:
# load treained model
loaded_model = joblib.load(filename)

In [0]:
# Open the input video file
input_movie = cv2.VideoCapture(files_path+"test_dir/input.mp4")

length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(input_movie.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = input_movie.get(cv2.CAP_PROP_FPS)
frame_rate = round(frame_rate,2)

In [25]:
frame_width,frame_height,frame_rate,length

(640, 480, 29.97, 841)

In [0]:
# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output_movie = cv2.VideoWriter(files_path+'test_dir/Output.mp4', fourcc, frame_rate, (frame_width, frame_height))

In [31]:
%time
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0
while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame,model="cnn")
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    face_locations_pred = []
    try:
        for i, face_encoding in enumerate(face_encodings):
            # See if the face is a match for the known face(s)
            prob = loaded_model.predict_proba([face_encoding])[0]
            
            if any([v*100 > 70 for v in prob]):
                # print(prob)
                face_names.append(loaded_model.predict([face_encoding])[0])
                face_locations_pred.append(face_locations[i])
        # Label the results
        for (top, right, bottom, left), name in zip(face_locations_pred, face_names):
            if not name:
                continue

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
    except Exception as ex:
        print(ex)
    finally:
        # Write the resulting image to the output video file
        print("Writing frame {} / {}".format(frame_number, length))
        output_movie.write(frame)

# All done!
input_movie.release()
output_movie.release()
cv2.destroyAllWindows()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Writing frame 1 / 841
Writing frame 2 / 841
Writing frame 3 / 841
Writing frame 4 / 841
Writing frame 5 / 841
Writing frame 6 / 841
Writing frame 7 / 841
Writing frame 8 / 841
Writing frame 9 / 841
Writing frame 10 / 841
Writing frame 11 / 841
Writing frame 12 / 841
Writing frame 13 / 841
Writing frame 14 / 841
Writing frame 15 / 841
Writing frame 16 / 841
Writing frame 17 / 841
Writing frame 18 / 841
Writing frame 19 / 841
Writing frame 20 / 841
Writing frame 21 / 841
Writing frame 22 / 841
Writing frame 23 / 841
Writing frame 24 / 841
Writing frame 25 / 841
Writing frame 26 / 841
Writing frame 27 / 841
Writing frame 28 / 841
Writing frame 29 / 841
Writing frame 30 / 841
Writing frame 31 / 841
Writing frame 32 / 841
Writing frame 33 / 841
Writing frame 34 / 841
Writing frame 35 / 841
Writing frame 36 / 841
Writing frame 37 / 841
Writing frame 38 / 841
Writing frame 39 / 841
Writing frame 40 / 841
Writing frame 41 / 841
Wr

## Reference
- Face Recognition [The world's simplest facial recognition api for Python and the command line](https://github.com/ageitgey/face_recognition)